Please run The following cell to start your two-hour session

In [ ]:
from new_session import *
start_session()

<h1>Lab 2: Visualization of Electric Field Symmetries and Gauss's Law</h1>

In this lab, you will be exploring the three symmetries for which Gauss's Law can be applied to calculate the electric field: spherical, cylindrical and planar symmetries. In this exploration, you will

<ol>
    <li> Visualize the electric field produced by uniformly-charged spheres, cylinders and slabs, both inside and outside of the charged source.</li>
    <li> Choose the appropriate shape, dimensions and location of Gaussian surface needed for each case.</li>
</ol>

We have extended the code you used and edited in Lab 1 so that it can plot the E field of symmetric continuous shapes. At the very core of the code, the calculate_E() function that you wrote is calculating the E field of small contributions along the shape and then they are added together, i.e. integrating the field over the volume of the shape.


In [ ]:
#To Analia and Paola, still debating whether to hide all these functions in a seprarte file and import them or just keep them.

from vpython import *

def calculate_E(q,r_S,r_P):
    e0 = 8.854187817e-12  
    k = 1.0 / (4 * pi * e0)
    rSP = sqrt(((r_P[0]-r_S[0])**2) + ((r_P[1]-r_S[1])**2) + ((r_P[2]-r_S[2])**2))
    E_x = k*q *( (r_P[0]-r_S[0]) / rSP**3) 
    E_y = k*q *( (r_P[1]-r_S[1]) / rSP**3) 
    E_z = k*q *( (r_P[2]-r_S[2]) / rSP**3)      
    return([E_x,E_y,E_z])


def draw_configuration(charges,loc,points,draw_contributions = True, mode = None):
    
    if mode !=None:
        draw_contributions = False
    for point in points:
        Es = [calculate_E(charges[index],loc[index],point) for index in range(len(charges))]
        E_tot = [sum(E) for E in zip(*Es)]
        colors = [color.blue if charge<0 else color.red for charge in charges]
        radii = [0.5e9*abs(charge) for charge in charges]
        if mode==None:
            for index in range(len(charges)):
                sphere(pos=vector(loc[index][0],loc[index][1],loc[index][2]), radius=radii[index], color=colors[index])
        Emagn = mag(vector(E_tot[0], E_tot[1],E_tot[2]))
        scale = 1
        if  Emagn >0:
            if mode !=None:
                scale = 1. / Emagn 
            E_tot = [e*scale for e in E_tot]
            if mode !=None and charges[0]<0:
                Earrow_tot=arrow(pos=vector(-E_tot[0]+point[0],-E_tot[1]+point[1],-E_tot[2]+point[2]), axis=vector(E_tot[0], E_tot[1],E_tot[2]), shaftwidth=0.05,color=color.green)
            else:
                Earrow_tot=arrow(pos=vector(point[0],point[1],point[2]), axis=vector(E_tot[0], E_tot[1],E_tot[2]), shaftwidth=0.05,color=color.green)
        if draw_contributions:
            for index in range(len(Es)):
                E = [e*scale for e in Es[index]]
                arrow(pos=vector(point[0],point[1],point[2]), axis=vector(E[0], E[1],E[2]),  shaftwidth=0.03,color=colors[index])


def draw_charged_sphere(charge=1e-9,radius=1,distance=2):
    scene = canvas() 
    col = color.blue 
    if charge>0:
        col = color.red 
    sphere(pos=vector(0,0,0), radius=radius, color=col,opacity = 0.5)
    if distance < radius:
        charge = charge*(distance**3)/(radius**3)
        
    d = distance/(sqrt(2))
    points = [[0,0,distance],[0,distance,0],[distance,0,0]]
    points+= [[0,0,-distance],[0,-distance,0],[-distance,0,0]]
    points+= [[0,d,d],[d,d,0],[d,0,d],[0,-d,-d],[-d,-d,0],[-d,0,-d]]
    points+= [[0,-d,d],[-d,d,0],[-d,0,d],[0,d,-d],[d,-d,0],[d,0,-d]]
    
    draw_configuration([charge],[[0,0,0]],points,draw_contributions = False, mode = "sphere")

def draw_charged_cylinder(lambda0=1e-9,length=10,radius=1,distance=2):
    scene = canvas() 
    s = "In VPython programs:\n"
    s += "    Rotate the camera by dragging with the right mouse button,\n        or hold down the Ctrl key and drag.\n"
    s += "    To zoom, drag with the left+right mouse buttons,\n         or hold down the Alt/Option key and drag,\n         or use the mouse wheel.\n"
    s += "Touch screen: pinch/extend to zoom, swipe or two-finger rotate."
    scene.caption = s
    col = color.blue 
    if lambda0>0:
        col = color.red 
    cylinder(pos=vector(-length/2,0,0),axis=vector(length,0,0), radius=radius, color=col,opacity = 0.5)
    if distance <radius:
        lambda0 = lambda0*(distance**2)/(radius**2)
    dx = length/1000
    charges = []
    locs = []
    for x in arange(-length/2, length/2, dx):
        charges.append(lambda0*dx)
        locs.append([x,0,0])
    d = distance/(sqrt(2))
    points = []
    for x in arange(-int(length/2), int(length/2)):
        points += [[x,0,distance],[x,distance,0]]

        points+= [[x,0,-distance],[x,-distance,0]]
        points+= [[x,d,d],[x,-d,-d]]
        points+= [[x,-d,d],[x,d,-d]]
    points+= [[-length/1.98,0,0],[0.05+length/1.98,0,0],[-length/1.6,0,0],[0.05+length/1.6,0,0]]
    draw_configuration(charges,locs,points,draw_contributions = False, mode = "cylinder")

def draw_charged_slab(rho=1e-9,length=10,width=1,height=10,distance=2):
    scene = canvas() 
    col = color.blue 
    if rho>0:
        col = color.red 
        
    box(pos=vector(0,0,0),length=length,height=height,width=width, color=col,opacity = 0.5)
    if distance <width/2:
        rho = 2*rho*(distance)/(width)
    dx = length/50
    dy = width/50
    charges = []
    locs = []
    for x in arange(-length/2, length/2, dx):
        for y in arange(-height/2,height/2,dy):
            charges.append(rho*distance*dx*dy)
            locs.append([x,y,0])
    d = distance/(sqrt(2))
    points = []
    for x in arange(-int(length/2), int(length/2)):
        for y in arange(-int(height/2), int(height/2)):
            points += [[x,y,distance],[x,y,-distance]]
    for y in arange(-int(height/2), int(height/2)):
        points+= [[-length/1.98,y,0],[0.05+length/1.98,y,0]]
    draw_configuration(charges,locs,points,draw_contributions = False, mode = "plane")

    
def draw_Gaussian_surface(mode,radius=1,circle_centers=[[0,0,0],[10,0,0]]):
    if mode =="sphere":
        sphere(pos=vector(0,0,0),radius=radius,color=color.white,opacity = 0.2)
    if mode =="cylinder":
        base = circle_centers[0]
        diff = [circle_centers[1][i]-base[i] for i in range(3)]
        cylinder(pos=vector(base[0],base[1],base[2]),axis=vector(diff[0],diff[1],diff[2]), radius=radius, color=color.white,opacity = 0.2)


<h2>Task 1: Visualize the Electric Field of a Uniformly Charged Insulating Sphere</h2>

Let's first explore how the electric field of a uniformly charged insulating sphere looks like in 3D. Use the cell below to run the visualization of the sphere. The function we built for a sphere takes three arguments:

<ol>
    <li>radius: radius of the sphere</li>
    <li>charge: total charge of the sphere</li>
    <li>distance: the distance from the origin of the sphere at which the E field is calculated and visualized</li>
</ol>

Please keep the charge in the nC order of magnitude (~1e-9 C) but experiment with all arguments to generate the E field for positive and negative spheres, both inside and outside of the sphere. Take screenshots for all four cases.

In [157]:
source_radius = 1
point_distance = 2
draw_charged_sphere(charge=1e-9,radius=source_radius,distance=point_distance)


<IPython.core.display.Javascript object>

In [158]:
draw_Gaussian_surface("sphere",radius = point_radius)


In [162]:
distance = 2
length = 6
radius = 1
draw_charged_cylinder(lambda0=1e-9,radius=radius,length=length,distance = distance)



<IPython.core.display.Javascript object>

In [163]:
draw_Gaussian_surface("cylinder",radius = distance,circle_centers=[[-1,0,0],[1,0,0]])


In [173]:
length = 6 #x
height = 6 #y
width = 2 #z
distance = 1.5 #above/below the center of the slab in z
draw_charged_slab(rho=1e-9,length=length,height=height,width=width,distance=distance)

<IPython.core.display.Javascript object>

In [174]:
draw_Gaussian_surface("cylinder",radius = 1,circle_centers=[[0,0,distance],[0,0,-distance]])
